In [ ]:
### In the code below(line 18, but double check) replace "Recovered" with "Confirmed" and "Deaths" to get confirmed and deaths cases respectively

In [1]:
#import all packeges
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
from datetime import timedelta
from sklearn.svm import SVR
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from sklearn.metrics import mean_squared_error,r2_score
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [4]:
#Read the dataset
covid=pd.read_csv("C:/Users/Abbas/Desktop/countries-aggregated.csv")
covid.head()

,Date,Country,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0,0
1,2020-01-22,Albania,0,0,0
2,2020-01-22,Algeria,0,0,0
3,2020-01-22,Andorra,0,0,0
4,2020-01-22,Angola,0,0,0


In [5]:
countries = covid['Country'].to_list()
countries

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Angola',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burma',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Diamond Princess',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 

In [ ]:
#Extracting India's data 
for i in countries:
    covid_india=covid[covid['Country']== i]
    covid_india.head()
    covid_india["ObservationDate"]=pd.to_datetime(covid_india["Date"])
    covid_india.head()
    india_datewise=covid_india.groupby(["ObservationDate"]).agg({"Confirmed":'sum',"Recovered":'sum',"Deaths":'sum'})
    india_datewise
    india_datewise["Days Since"]=(india_datewise.index-india_datewise.index[0])
    india_datewise["Days Since"]=india_datewise["Days Since"].dt.days
    india_datewise
#training, testing
    train_ml=india_datewise.iloc[:int(india_datewise.shape[0]*0.95)]
    valid_ml=india_datewise.iloc[int(india_datewise.shape[0]*0.95):]
#giving hyperparameters
    svm=SVR(degree=7,kernel='poly')
#fitting Days and Confirmed cases in model
    svm.fit(np.array(train_ml["Days Since"]).reshape(-1,1),train_ml["Recovered"])
#MSE
    prediction_svm=svm.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))
    print("Root Mean Square Error for SVR Model for" +i+ ":" ,np.sqrt(mean_squared_error(prediction_svm,valid_ml["Recovered"])))
    predictions=svm.predict(np.array(india_datewise["Days Since"]).reshape(-1,1))
#Adding new dates
    new_date=[]
    new_prediction_svm=[]
    for i in range(1,8):
        new_date.append(india_datewise.index[-1]+timedelta(days=i))
        new_prediction_svm.append(svm.predict(np.array(india_datewise["Days Since"].max()+i).reshape(-1,1))[0])
    
#Model Prediction dataFrame
    model_predictions=pd.DataFrame(zip(new_date,new_prediction_svm),columns=["Date","SVM Prediction for Recovered"])
    model_predictions.tail(1)
    print(model_predictions.tail(1))
# ##fitting Days and Confirmed cases in model
#     svm.fit(np.array(train_ml["Days Since"]).reshape(-1,1),train_ml["Deaths"])
# #MSE
#     prediction_svm=svm.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))
#     print("Root Mean Square Error for SVR Model:" +i+ ":",np.sqrt(mean_squared_error(prediction_svm,valid_ml["Deaths"])))
#     predictions=svm.predict(np.array(india_datewise["Days Since"]).reshape(-1,1))
#     #Adding new date
#     new_date=[]
#     new_prediction_svm=[]
#     for i in range(1,18):
#         new_date.append(india_datewise.index[-1]+timedelta(days=i))
#         new_prediction_svm.append(svm.predict(np.array(india_datewise["Days Since"].max()+i).reshape(-1,1))[0])
# #Dataframe for deaths prediction
#     model_predictions=pd.DataFrame(zip(new_date,new_prediction_svm),columns=["Date","SVM Prediction for Deaths"])
#     model_predictions.head()
#     print(model_predictions)
# #Applied same steps for recovered
#     svm.fit(np.array(train_ml["Days Since"]).reshape(-1,1),train_ml["Recovered"])
#     prediction_svm=svm.predict(np.array(valid_ml["Days Since"]).reshape(-1,1))
#     print("Root Mean Square Error for SVR Model" +i+ ":",np.sqrt(mean_squared_error(prediction_svm,valid_ml["Recovered"])))
#     #Adding new date
#     new_date=[]
#     new_prediction_svm=[]
#     for i in range(1,18):
#         new_date.append(india_datewise.index[-1]+timedelta(days=i))
#         new_prediction_svm.append(svm.predict(np.array(india_datewise["Days Since"].max()+i).reshape(-1,1))[0])
#     model_predictions=pd.DataFrame(zip(new_date,new_prediction_svm),columns=["Date","SVM Prediction for Recovered"])
#     model_predictions.head()
#     print(model_predictions)

Root Mean Square Error for SVR Model forAfghanistan: 88.26825392067171
        Date  SVM Prediction for Recovered
6 2020-05-01                     146.51003
Root Mean Square Error for SVR Model forAlbania: 83.88420526198195
        Date  SVM Prediction for Recovered
6 2020-05-01                    850.996429
Root Mean Square Error for SVR Model forAlgeria: 31.535805447236903
        Date  SVM Prediction for Recovered
6 2020-05-01                   2368.309818
Root Mean Square Error for SVR Model forAndorra: 65.21193340265833
        Date  SVM Prediction for Recovered
6 2020-05-01                    460.376479
Root Mean Square Error for SVR Model forAngola: 1.8861048425783948
        Date  SVM Prediction for Recovered
6 2020-05-01                     14.891252
Root Mean Square Error for SVR Model forAntigua and Barbuda: 8.159457412793833
        Date  SVM Prediction for Recovered
6 2020-05-01                      1.130116
Root Mean Square Error for SVR Model forArgentina: 187.8218670424

Root Mean Square Error for SVR Model forEl Salvador: 14.834036793389092
        Date  SVM Prediction for Recovered
6 2020-05-01                     89.422925
Root Mean Square Error for SVR Model forEquatorial Guinea: 1.123078372331362
        Date  SVM Prediction for Recovered
6 2020-05-01                     14.982468
Root Mean Square Error for SVR Model forEritrea: 6.1761408323287075
        Date  SVM Prediction for Recovered
6 2020-05-01                      1.491423
Root Mean Square Error for SVR Model forEstonia: 35.08900373300675
        Date  SVM Prediction for Recovered
6 2020-05-01                    418.196483
Root Mean Square Error for SVR Model forEswatini: 3.7198584911798873
        Date  SVM Prediction for Recovered
6 2020-05-01                     23.209162
Root Mean Square Error for SVR Model forEthiopia: 1.8577330240133116
        Date  SVM Prediction for Recovered
6 2020-05-01                     40.487311
Root Mean Square Error for SVR Model forFiji: 7.01427116670007